# Zona de Importaciones

In [ ]:
import pandas as pd
import numpy as np
import random
import shutil
import tempfile
import yt_dlp
#from pytube import YouTube
import speech_recognition as sr
import whisper
from pydub import AudioSegment
import dask
from dask import delayed
from dask.diagnostics import ProgressBar
import dask.dataframe as dd
import time
# import concurrent.futures
import os
import re

from vosk import Model, KaldiRecognizer
# import deepspeech
import wave
import json

import subprocess

from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch
import librosa

import torchaudio
# from espnet_model_zoo.downloader import ModelDownloader

import re
import os

# os.environ["KERAS_BACKEND"] = "tensorflow"

# from glob import glob
import tensorflow as tf
import keras
from keras import layers

# pattern_wav_name = re.compile(r'([^/\\\.]+)')


# Variables de entorno

In [ ]:
COOKIES_FILE_PATH = '../conf/youtube.com_cookies.txt'
INPUT_FILE_PATH = '../data/01_raw/canciones_urls.xlsx'
OUTPUT_PATH = '../data/01_raw/'
AUDIO_PATH = '../data/01_raw/audios/'
LYRICS_FILE = '../data/01_raw/canciones_letra.xlsx'


# Lista de User-Agents para rotar
USER_AGENTS = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
]

# Cargar Dataframe

In [ ]:
df = pd.read_excel(INPUT_FILE_PATH)
df_lyrics = pd.read_excel(LYRICS_FILE)

In [ ]:
df_lyrics = df_lyrics[['AUDIO_NAME','LETRA','AUDIO_DESCARGADO']].copy()
df_manual_lyrics = df[['TITULO','AÑO','COMPOSITOR','UBICACIÓN','DEPARTAMENTO','URL','GENERO','LETRA']].copy()
df = df[['TITULO','AÑO','COMPOSITOR','UBICACIÓN','DEPARTAMENTO','URL','GENERO']].copy()

In [ ]:
df = df[pd.notna(df['URL'])]
df_manual_lyrics = df_manual_lyrics.dropna().copy()
df_lyrics = df_lyrics.dropna().copy()

In [ ]:
df['AUDIO_NAME'] = df['TITULO']+'_'+df['AÑO'].astype(str)+'_'+df['COMPOSITOR']
df_manual_lyrics['AUDIO_NAME'] = df_manual_lyrics['TITULO']+'_'+df_manual_lyrics['AÑO'].astype(str)+'_'+df_manual_lyrics['COMPOSITOR']
df_manual_lyrics['AUDIO_DESCARGADO'] = True

In [ ]:
df = df.merge(df_lyrics[['AUDIO_NAME','LETRA','AUDIO_DESCARGADO']], on='AUDIO_NAME', how='left')

In [ ]:
df = pd.concat([df, df_manual_lyrics], axis=0)
df.drop_duplicates(inplace=True)

# Buscar URL de Youtube

In [ ]:
def buscar_youtube_url(titulo: str, compositor: str)->str:
  """
    Función encargada de buscar una URL de YouTube según una query
    Args:
      titulo: str. Titulo de la canción
      compositor: str. Nombre del compositor
  """
  query = f'Canción: {titulo} - Compositor: {compositor}'
  # Crear una copia temporal del archivo de cookies
  temp_cookies = tempfile.NamedTemporaryFile(delete=False)
  shutil.copy(COOKIES_FILE_PATH, temp_cookies.name)
  
  ydl_opts = {
    "quiet": True,
    'geo_bypass': True,
    'cookiefile': temp_cookies.name
    }
  
  with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    try:
      result = ydl.extract_info(f"ytsearch:{query}", download=False)
      for entry in result["entries"]:
        if (entry.get("duration") and entry["duration"] > 60) and (entry.get("duration") and entry["duration"] < 480):
          return entry["webpage_url"]
      return result["webpage_url"]
    except Exception as e:
      print(f'Ocurrió un error en la búsqueda de la URL en YouTube: {e}')
      return ''

In [ ]:
df['URL'] = df.apply(lambda row: buscar_youtube_url(row['TITULO'], row['COMPOSITOR']))

# Exportar resultados

In [ ]:
df.to_excel('../data/02_intermediate/canciones_con_urls.xlsx', index=False)